In [ ]:
#| hide
#| default_exp script

# Script - CLI

> A fast way to turn your python function into a script.

Part of [fast.ai](https://www.fast.ai)'s toolkit for delightful developer experiences.

## Overview

Sometimes, you want to create a quick script, either for yourself, or for others. But in Python, that involves a whole lot of boilerplate and ceremony, especially if you want to support command line arguments, provide help, and other niceties. You can use [argparse](https://docs.python.org/3/library/argparse.html) for this purpose, which comes with Python, but it's complex and verbose.

`fastcore.script` makes life easier. There are much fancier modules to help you write scripts (we recommend [Python Fire](https://github.com/google/python-fire), and [Click](https://click.palletsprojects.com/en/7.x/) is also popular), but fastcore.script is very fast and very simple. In fact, it's <50 lines of code! Basically, it's just a little wrapper around `argparse` that uses modern Python features and some thoughtful defaults to get rid of the boilerplate.

For full details, see the [docs](https://fastcore.script.fast.ai) for `core`.

## Example

Here's a complete example (available in `examples/test_fastcore.py`):

```python
from fastcore.script import *
@call_parse
def main(msg:str,     # The message
         upper:bool): # Convert to uppercase?
    "Print `msg`, optionally converting to uppercase"
    print(msg.upper() if upper else msg)
````

If you copy that info a file and run it, you'll see:

```
$ examples/test_fastcore.py --help
usage: test_fastcore.py [-h] [--upper] msg

Print `msg`, optionally converting to uppercase

positional arguments:
  msg          The message

optional arguments:
  -h, --help   show this help message and exit
  --upper      Convert to uppercase? (default: False)
```

As you see, we didn't need any `if __name__ == "__main__"`, we didn't have to parse arguments, we just wrote a function, added a decorator to it, and added some annotations to our function's parameters. As a bonus, we can also use this function directly from a REPL such as Jupyter Notebook - it's not just for command line scripts!

You should provide a default (after the `=`) for any *optional* parameters. If you don't provide a default for a parameter, then it will be a *positional* parameter.

::: {.callout-warning}
## Boolean Arguments Default to False
Arguments of type `bool` or `store_true` default to `False` regardless of whether you provide a default or not. Use `bool_arg` as the type instead of `bool` if you want to set a default value of True. For example:

```python
@call_parse
def main(msg:str="Hi",     # The message
         upper:bool_arg=True): # Convert to uppercase?
```
:::

## Param annotations

If you want to use the full power of `argparse`, you can do so by using `Param` annotations instead of type annotations and [docments](https://fastcore.fast.ai/docments.html), like so:

```python
from fastcore.script import *
@call_parse
def main(msg:Param("The message", str),
         upper:Param("Convert to uppercase?", store_true)):
    "Print `msg`, optionally converting to uppercase"
    print(msg.upper() if upper else msg)
````

If you use this approach, then each parameter in your function should have an annotation `Param(...)` (as in the example above). You can pass the following when calling `Param`: `help`,`type`,`opt`,`action`,`nargs`,`const`,`choices`,`required` . Except for `opt`, all of these are just passed directly to `argparse`, so you have all the power of that module at your disposal. Generally you'll want to pass at least `help` (since this is provided as the help string for that parameter) and `type` (to ensure that you get the type of data you expect). `opt` is a bool that defines whether a param is optional or required (positional) - but you'll generally not need to set this manually, because fastcore.script will set it for you automatically based on *default* values.

## setuptools scripts

There's a really nice feature of pip/setuptools that lets you create commandline scripts directly from functions, makes them available in the `PATH`, and even makes your scripts cross-platform (e.g. in Windows it creates an exe). fastcore.script supports this feature too. The trick to making a function available as a script is to add a `console_scripts` section to your setup file, of the form: `script_name=module:function_name`. E.g. in this case we use: `test_fastcore.script=fastcore.script.test_cli:main`. With this, you can then just type `test_fastcore.script` at any time, from any directory, and your script will be called (once it's installed using one of the methods below).

You don't actually have to write a `setup.py` yourself. Instead, just use [nbdev](https://nbdev.fast.ai). Then modify `settings.ini` as appropriate for your module/script. To install your script directly, you can type `pip install -e .`. Your script, when installed this way (it's called an [editable install](http://codumentary.blogspot.com/2014/11/python-tip-of-year-pip-install-editable.html)), will automatically be up to date even if you edit it - there's no need to reinstall it after editing. With nbdev you can even make your module and script available for installation directly from pip and conda by running `make release`.

## API details

In [ ]:
#| export
import inspect,argparse,shutil,types

from functools import wraps,partial
from fastcore.imports import *
from fastcore.utils import *
from fastcore.docments import docments
from typing import get_origin, get_args, Union

In [ ]:
#| hide
from fastcore.test import *

In [ ]:
#| export
def store_true():
    "Placeholder to pass to `Param` for `store_true` action"
    pass

In [ ]:
#| export
def store_false():
    "Placeholder to pass to `Param` for `store_false` action"
    pass

In [ ]:
#| export
def bool_arg(v):
    "Use as `type` for `Param` to get `bool` behavior"
    return str2bool(v)

In [ ]:
#| export
def clean_type_str(x:str):
    x = str(x)
    x = re.sub(r"(enum |class|function|__main__\.|\ at.*)", '', x)
    x = re.sub(r"(<|>|'|\ )", '', x) # spl characters
    return x

In [ ]:
class Test: pass

test_eq(clean_type_str(argparse.ArgumentParser), 'argparse.ArgumentParser')
test_eq(clean_type_str(Test), 'Test')
test_eq(clean_type_str(int), 'int')
test_eq(clean_type_str(float), 'float')
test_eq(clean_type_str(store_false), 'store_false')

## Param -

In [ ]:
#| export
class Param:
    "A parameter in a function used in `anno_parser` or `call_parse`"
    def __init__(self, help="", type=None, opt=True, action=None, nargs=None, const=None,
                 choices=None, required=None, default=None, version=None):
        if type==store_true:  type,action,default=None,'store_true',False
        if type==store_false: type,action,default=None,'store_false',True
        if type and isinstance(type,typing.Type) and issubclass(type,enum.Enum) and not choices: choices=list(type)
        help = help or ""
        self.negated = False
        store_attr()

    def set_default(self, d):
        if self.action == "version":
            if d != inspect.Parameter.empty: self.version = d
            self.opt = True
            return
        if self.type==bool and self.action is None:
            self.type = None
            if d is True: self.action,self.default,self.negated = 'store_false',True,True
            else:         self.action,self.default = 'store_true',False
        elif self.default is None:
            if d == inspect.Parameter.empty: self.opt = False
            else: self.default = d
        if self.default is not None: self.help += f" (default: {self.default})"

    @property
    def pre(self): return '--' if self.opt else ''
    @property
    def kwargs(self): return {k:v for k,v in self.__dict__.items() if v is not None and k not in ('opt','negated') and k[0]!='_'}
    def __repr__(self):
        if not self.help and self.type is None: return ""
        if not self.help and self.type is not None: return f"{clean_type_str(self.type)}"
        if self.help and self.type is None: return f"<{self.help}>"
        if self.help and self.type is not None: return f"{clean_type_str(self.type)} <{self.help}>"

In [ ]:
test_eq(repr(Param("Help goes here")), '<Help goes here>')
test_eq(repr(Param("Help", int)), 'int <Help>')
test_eq(repr(Param(help=None, type=int)), 'int')
test_eq(repr(Param(help=None, type=None)), '')

Each parameter in your function should have an annotation `Param(...)`. You can pass the following when calling `Param`: `help`,`type`,`opt`,`action`,`nargs`,`const`,`choices`,`required`, `version` (i.e. it takes the same parameters as `argparse.ArgumentParser.add_argument`, plus `opt`). Except for `opt`, all of these are just passed directly to `argparse`, so you have all the power of that module at your disposal. Generally you'll want to pass at least `help` (since this is provided as the help string for that parameter) and `type` (to ensure that you get the type of data you expect).

`opt` is a bool that defines whether a param is optional or required (positional) - but you'll generally not need to set this manually, because fastcore.script will set it for you automatically based on *default* values. You should provide a default (after the `=`) for any *optional* parameters. If you don't provide a default for a parameter, then it will be a *positional* parameter.

Param's `__repr__` also allows for more informative function annotation when looking up the function's doc using shift+tab. You see the type annotation (if there is one) and the accompanying help documentation with it.

In [ ]:
def f(required:Param("Required param", int),
      a:Param("param 1", bool_arg),
      b:Param("param 2", str)="test"):
    "my docs"
    ...

In [ ]:
help(f)

Help on function f in module __main__:

f(required: int <Required param>, a: bool_arg <param 1>, b: str <param 2> = 'test')
    my docs



In [ ]:
p = Param(help="help", type=int)
p.set_default(1)
test_eq(p.kwargs, {'help': 'help (default: 1)', 'type': int, 'default': 1})

## anno_parser -

In [ ]:
#| export
class _HelpFormatter(argparse.HelpFormatter):
    def __init__(self, prog, indent_increment=2):
        cols = shutil.get_terminal_size((120,30))[0]
        super().__init__(prog, max_help_position=cols//2, width=cols, indent_increment=indent_increment)
    def _expand_help(self, action): return self._get_help_string(action)

In [ ]:
#| export
def _is_union(t): return get_origin(t) in (Union, types.UnionType) if hasattr(types, 'UnionType') else get_origin(t) is Union

def _union_parser(types):
    "Return a parser that tries each type in sequence"
    def _parse(v):
        for t in types:
            if t is type(None): continue
            try: return t(v)
            except: pass
        raise ValueError(f"Could not parse {v!r} as any of {types}")
    return _parse

def _union_type(t):
    "Get parser for Union types, or None if not a Union"
    if not _is_union(t): return None
    return _union_parser(get_args(t))

In [ ]:
#| hide
test_eq(_is_union(str | list), True)
test_eq(_is_union(int), False)
test_eq(_union_parser((int, str))('42'), 42)
test_eq(_union_parser((int, str))('hello'), 'hello')
test_eq(_union_type(int | str)('42'), 42)
test_eq(_union_type(int | str)('abc'), 'abc')
test_eq(_union_type(int), None)

In [ ]:
#| export
def anno_parser(func, prog:str=None):
    "Look at params (annotated with `Param`) in func and return an `ArgumentParser`"
    p = argparse.ArgumentParser(description=func.__doc__, prog=prog, formatter_class=_HelpFormatter)
    for k,v in docments(func, full=True, returns=False, eval_str=True).items():
        param = v.anno
        if not isinstance(param, Param):
            anno = _union_type(v.anno) or v.anno
            param = Param(v.docment, anno)
        param.set_default(v.default)
        name = f"no-{k}" if param.negated else k
        kw = param.kwargs
        if param.negated: kw['dest'] = k
        p.add_argument(f"{param.pre}{name}", **kw)
    p.add_argument(f"--pdb", help=argparse.SUPPRESS, action='store_true')
    p.add_argument(f"--xtra", help=argparse.SUPPRESS, type=str)
    return p

This converts a function with parameter annotations of type `Param` into an `argparse.ArgumentParser` object. Function arguments with a default provided are optional, and other arguments are positional.

In [ ]:
_en = str_enum('_en', 'aa','bb','cc')
def f(required:Param("Required param", int),
      a:Param("param 1", bool_arg),
      v:Param("Print version", action='version', version='%(prog)s 2.0.0'),
      b:Param("param 2", str)="test",
      c:Param("param 3", _en)=_en.aa):
    "my docs"
    ...

p = anno_parser(f, 'progname')
p.print_help()

usage: progname [-h] [--v] [--b B] [--c {aa,bb,cc}] required a

my docs

positional arguments:
  required        Required param
  a               param 1

options:
  -h, --help      show this help message and exit
  --v             Print version
  --b B           param 2 (default: test)
  --c {aa,bb,cc}  param 3 (default: aa)


We can also check the version and help flags are working.

In [ ]:
try: p.parse_args(['--v'])
except: pass

progname 2.0.0


In [ ]:
try: p.parse_args(['-h'])
except: pass

usage: progname [-h] [--v] [--b B] [--c {aa,bb,cc}] required a

my docs

positional arguments:
  required        Required param
  a               param 1

options:
  -h, --help      show this help message and exit
  --v             Print version
  --b B           param 2 (default: test)
  --c {aa,bb,cc}  param 3 (default: aa)


It also works with type annotations and docments:

In [ ]:
def g(required:int,  # Required param
      a:bool_arg,    # param 1
      b="test",      # param 2
      c:_en=_en.aa): # param 3
    "my docs"
    ...

p = anno_parser(g, 'progname')
p.print_help()

usage: progname [-h] [--b B] [--c {aa,bb,cc}] required a

my docs

positional arguments:
  required        Required param
  a               param 1

options:
  -h, --help      show this help message and exit
  --b B           param 2 (default: test)
  --c {aa,bb,cc}  param 3 (default: aa)


It also works with Union types:

In [ ]:
def h(n:int|str, exts:str|list=None):
    "Test union types"

p = anno_parser(h, 'test')
p.print_help()

usage: test [-h] [--exts EXTS] n

Test union types

positional arguments:
  n

options:
  -h, --help   show this help message and exit
  --exts EXTS


In [ ]:
test_eq(p.parse_args(['42', '--exts', 'py']).n, 42)
test_eq(p.parse_args(['hello']).n, 'hello')

Sometimes it's convenient to extract arguments from the actual name of the called program. `args_from_prog` will do this, assuming that names and values of the params are separated by a `#`. Optionally there can also be a prefix separated by `##` (double underscore).

In [ ]:
#| export
def args_from_prog(func, prog):
    "Extract args from `prog`"
    if prog is None or '#' not in prog: return {}
    if '##' in prog: _,prog = prog.split('##', 1)
    progsp = prog.split("#")
    args = {progsp[i]:progsp[i+1] for i in range(0, len(progsp), 2)}
    annos = type_hints(func)
    for k,v in args.items():
        anno = annos.get(k)
        t = getattr(anno, 'type', anno)
        if t in (bool, bool_arg): t = str2bool
        elif isinstance(anno, Param) and anno.action in ('store_true', 'store_false'): t = str2bool
        if t: args[k] = t(v)
    return args

In [ ]:
exp = {'a': False, 'b': 'baa'}
test_eq(args_from_prog(f, 'foo##a#0#b#baa'), exp)
test_eq(args_from_prog(f, 'a#0#b#baa'), exp)

In [ ]:
#| export
SCRIPT_INFO = SimpleNamespace(func=None)

## call_parse -

In [ ]:
#| export
from contextvars import ContextVar
from contextlib import contextmanager

In [ ]:
#| export
@contextmanager
def set_ctx(cv, val=True):
    token = cv.set(val)
    try: yield
    finally: cv.reset(token)

In [ ]:
#| export
_in_call_parse = ContextVar('_in_call_parse', default=False)

def call_parse(func=None, nested=False):
    "Decorator to create a simple CLI from `func` using `anno_parser`"
    if func is None: return partial(call_parse, nested=nested)

    @wraps(func)
    def _f(*args, **kwargs):
        if args or kwargs or _in_call_parse.get(): return func(*args, **kwargs)
        with set_ctx(_in_call_parse):
            mod = inspect.getmodule(inspect.currentframe().f_back)
            if not mod: return func(*args, **kwargs)
            if not SCRIPT_INFO.func and mod.__name__=="__main__": SCRIPT_INFO.func = func.__name__
            if len(sys.argv)>1 and sys.argv[1]=='': sys.argv.pop(1)
            p = anno_parser(func)
            if nested: args, sys.argv[1:] = p.parse_known_args()
            else: args = p.parse_args()
            args = args.__dict__
            xtra = otherwise(args.pop('xtra', ''), eq(1), p.prog)
            tfunc = trace(func) if args.pop('pdb', False) else func
            return tfunc(**merge(args, args_from_prog(func, xtra)))

    mod = inspect.getmodule(inspect.currentframe().f_back)
    if getattr(mod, '__name__', '') =="__main__":
        setattr(mod, func.__name__, _f)
        SCRIPT_INFO.func = func.__name__
        return _f()
    else: return _f

In [ ]:
@call_parse
def test_add(
    a:int=0,  # param a
    b:int=0  # param 1
):
    "Add up `a` and `b`"
    return a + b

`call_parse` decorated functions work as regular functions and also as command-line interface functions.

In [ ]:
test_eq(test_add(1,2), 3)

This is the main way to use `fastcore.script`; decorate your function with `call_parse`, add `Param` annotations (as shown above) or type annotations and docments, and it can then be used as a script.

Use the `nested` keyword argument to create nested parsers, where earlier parsers consume only their known args from `sys.argv` before later parsers are used. This is useful to create one command line application that executes another. For example:

```sh
myrunner --keyword 1 script.py -- <script.py args>
```

A separating `--` after the first application's args is recommended though not always required, otherwise args may be parsed in unexpected ways. For example:

```sh
myrunner script.py -h
```

would display `myrunner`'s help and not `script.py`'s.

## Export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()